In [37]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from textblob import TextBlob
import ast

In [38]:
# cargar los archivos csv en dataframes de pandas
df_games = pd.read_csv('231_osg.csv', encoding='utf-8')
df_reviews = pd.read_csv('232_aur.csv', encoding='utf-8')
df_items = pd.read_csv('233_aui.csv', encoding='utf-8')


In [39]:
# terminar de transformar algunos datos
df_games['año_lanzamiento'].fillna(0, inplace=True)
df_games['año_lanzamiento'] = df_games['año_lanzamiento'].astype(int)

# CREACION DE COLUMNA sentiment_analysis

## Consigna

En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1

## Creacion y aplicacion de la funcion

In [40]:
# definir funcion que se aplicara a la columna reviews_review
def analizar_sentimiento(comentario):
    if pd.isnull(comentario) or comentario.lower() == 'sin comentario':
        return 1  # Neutral o sin review
    else:
        analysis = TextBlob(comentario)
        if analysis.sentiment.polarity < 0:
            return 0  # Malo
        elif analysis.sentiment.polarity == 0:
            return 1  # Neutral
        else:
            return 2  # Positivo


In [41]:

# aplicar la función a la columna reviews_review creando la columna sentiment_analysis
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(analizar_sentimiento)


In [42]:

# verificar la creacion de la columna sentiment_analysis
df_reviews

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud,1
...,...,...,...,...,...,...,...,...
59031,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,"Posted August 15, 2014.",440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2
59032,76561198141079508,http://steamcommunity.com/profiles/76561198141...,"Posted August 2, 2014.",304930,No ratings yet,True,Fun game with friends,0
59033,ShadowYT100,http://steamcommunity.com/id/ShadowYT100,"Posted July 31, 2015.",265630,No ratings yet,True,So Fun!! :D,2
59034,bestcustomurlevermade,http://steamcommunity.com/id/bestcustomurlever...,"Posted December 20, 2015.",304050,No ratings yet,True,"This game is great. The only thing is,Why cant...",2


Se Elimina la columna 'reviews_review'

## Elimninacion de la columna reviews_review

In [43]:
# eliminar la columna reviews_review
df_reviews = df_reviews.drop(columns=['reviews_review'])


In [44]:
# informacion general del dataframe
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59036 entries, 0 to 59035
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59036 non-null  object
 1   user_url            59036 non-null  object
 2   reviews_posted      59036 non-null  object
 3   reviews_item_id     59036 non-null  int64 
 4   reviews_helpful     59036 non-null  object
 5   reviews_recommend   59036 non-null  bool  
 6   sentiment_analysis  59036 non-null  int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 2.8+ MB


# CREACION DE ARCHIVOS A USARSE EN LAS 3 PRIMERAS FUNCIONES

## Archivos para la funcion 1 (developer): 

### Forma de la funcion

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:
Año,	    Cantidad de Items,	Contenido Free:
2023,	50,	                27%;
2022,	45,	                25%;
xxxx,	xx,	                xx%

### Obtencion de las columnas necesarias

In [45]:
a = df_games[['price', 'año_lanzamiento', 'developer', 'id']]
# renombrar columna 'id' para unirla con el dataframe anterior
a = a.rename(columns={'id':'item_id'})
# eliminar duplicados
print(a.duplicated().sum())
a = a.drop_duplicates()
print(a.duplicated().sum())
a

42702
0


,price,año_lanzamiento,developer,item_id
0,4.99,2018,Kotoshiro,761140
5,0.00,2018,Secret Level SRL,643980
9,0.00,2017,Poolians.com,670290
14,0.99,2017,彼岸领域,767400
17,3.99,2018,Trickjump Games Ltd,772540
...,...,...,...,...
71536,1.99,2018,Bidoniera Games,745400
71540,1.99,2018,"Nikita ""Ghost_RUS""",773640
71544,4.99,2018,Sacada,733530
71547,1.99,2018,Laush Dmitriy Sergeevich,610660


### Conversion a csv y parquet

In [46]:
a.to_csv('341a.csv', index=False, encoding='utf-8')
pq.write_table(pa.Table.from_pandas(a), '3410a.parquet')

## Archivo para la funcion 1 (developer): 
### Forma de la funcion 
### Obtencion de las columnas necesarias
### Conversion a csv y parquet

## Archivos para la funcion 2 (userdata): 

### Forma de la funcion 


def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

### Obtencion de las columnas necesarias


In [47]:
# selecionar 3 columnas de df_items
b = df_items[['items_count', 'user_id', 'item_id']]

# seleccionar 2 columnas de df_games
price_juegos = df_games[['price', 'id']]
# eliminar duplicados
price_juegos = price_juegos.drop_duplicates(subset='id', keep='first')
# renombrar 
price_juegos = price_juegos.rename(columns={'id':'item_id'})
price_juegos

# unir dataframes
b = b.merge(price_juegos, on='item_id', how='left')

# ver si hay nulos
print(b[b['price'].isnull()])
# imputar valores nulos con 0.0
df_relleno = b['price'].fillna(0.0)
# Se borra la columna original y se concatena la columna rellena con todo el dataframe
b = pd.concat([b.drop('price', axis=1), df_relleno], axis=1)

# eliminar columna item_id
b = b.drop('item_id', axis=1)
# agrupar por usuario sumando los precios de los juegos consumidos que es la cantidad gastada 
b_group = b.groupby('user_id')['price'].sum().reset_index()
b_group

# elegir la cantidad de items consumidos por cada usuario
df_count_items = b[['items_count', 'user_id']]
# eliminar duplicados
df_count_items = df_count_items.drop_duplicates(subset='user_id', keep='first')
# combinar dataframes
b = df_count_items.merge(b_group, on='user_id', how='right')
b

         items_count            user_id  item_id  price
13               277  76561197970982479    34440    NaN
26               277  76561197970982479     9340    NaN
27               277  76561197970982479   228200    NaN
37               277  76561197970982479    17340    NaN
51               277  76561197970982479    23120    NaN
...              ...                ...      ...    ...
5094089          177  76561198326700687   508380    NaN
5094091          177  76561198326700687   510660    NaN
5094095          177  76561198326700687   519170    NaN
5094101            7  76561198329548331   373330    NaN
5094104            7  76561198329548331   519140    NaN

[937657 rows x 4 columns]


,items_count,user_id,price
0,58,--000--,397.78
1,44,--ace--,166.82
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,427.50
4,149,-404PageNotFound-,1509.32
...,...,...,...
70907,5,zzonci,19.98
70908,61,zzoptimuszz,64.98
70909,13,zzydrax,99.94
70910,84,zzyfo,828.51


### Conversion a csv y parquet

In [48]:
b.to_csv('342b.csv', index=False, encoding='utf-8')
pq.write_table(pa.Table.from_pandas(b), '3420b.parquet')

## Archivos para la funcion 3 (UserForGenre): 



### Forma de la funcion 



def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}


### Obtencion de las columnas necesarias


In [49]:
# seleccionar 3 columnas de df_items
playtime_forever_usuario_item = df_items[['playtime_forever', 'user_id', 'item_id']]
playtime_forever_usuario_item
# seleccionar 2 columnas de df_games
genre_item = df_games[['genres','año_lanzamiento','id']]
# renombrar columna 'id' para unirla con el dataframe anterior
genre_item = genre_item.rename(columns={'id':'item_id'})
genre_item

# combinar dataframes
df_playtime_genre_aux = playtime_forever_usuario_item.merge(genre_item, on='item_id')
# eliminar columna item_id
df_playtime_genre= df_playtime_genre_aux.drop(columns=['item_id'])
df_playtime_genre
# Se transforma la columna 'playtime_forever' a horas
df_playtime_genre['playtime_horas'] = df_playtime_genre['playtime_forever']/60
df_playtime_genre
# asignar tipo de dato entero a la columna play_time
df_playtime_genre['playtime_horas'] = df_playtime_genre['playtime_horas'].astype('int32')
# verificar la conversion de tipo de dato
df_playtime_genre.info()
# Agrupar por user_id, año_lanzamiento y sumar las horas jugadas
df_playtime_genre2 = df_playtime_genre.groupby(['user_id','genres','año_lanzamiento'])['playtime_horas'].sum().reset_index()
df_playtime_genre2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9877304 entries, 0 to 9877303
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   playtime_forever  int64 
 1   user_id           object
 2   genres            object
 3   año_lanzamiento   int32 
 4   playtime_horas    int32 
dtypes: int32(2), int64(1), object(2)
memory usage: 301.4+ MB


,user_id,genres,año_lanzamiento,playtime_horas
0,--000--,Action,0,0
1,--000--,Action,2009,88
2,--000--,Action,2010,0
3,--000--,Action,2011,108
4,--000--,Action,2012,1820
...,...,...,...,...
3495277,zzzmidmiss,Strategy,2009,0
3495278,zzzmidmiss,Strategy,2010,0
3495279,zzzmidmiss,Strategy,2011,1
3495280,zzzmidmiss,Strategy,2012,14


### Conversion a csv y parquet

In [50]:
c = pd.read_csv('343c.csv', encoding='utf-8')
pq.write_table(pa.Table.from_pandas(c), '3430c.parquet')

## Archivos para las funciones 2 y 5: 


### Obtencion de las columnas necesarias


In [51]:
# seleccionar año_lanzamiento del df_games
anio_lanzamiento_item = df_games[['id', 'año_lanzamiento']]
anio_lanzamiento_item

,id,año_lanzamiento
0,761140,2018
1,761140,2018
2,761140,2018
3,761140,2018
4,761140,2018
...,...,...
71547,610660,2018
71548,610660,2018
71549,610660,2018
71550,658870,2017


In [52]:
# renombrar columna id para especificarla como columna clave en el merge
anio_lanzamiento_item = anio_lanzamiento_item.rename(columns={'id':'reviews_item_id'})
# eliminar duplicados
anio_lanzamiento_item = anio_lanzamiento_item.drop_duplicates()
df_reviews

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,2
2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,2
3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,1
...,...,...,...,...,...,...,...
59031,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,"Posted August 15, 2014.",440,No ratings yet,True,2
59032,76561198141079508,http://steamcommunity.com/profiles/76561198141...,"Posted August 2, 2014.",304930,No ratings yet,True,0
59033,ShadowYT100,http://steamcommunity.com/id/ShadowYT100,"Posted July 31, 2015.",265630,No ratings yet,True,2
59034,bestcustomurlevermade,http://steamcommunity.com/id/bestcustomurlever...,"Posted December 20, 2015.",304050,No ratings yet,True,2


In [53]:
# realizar merge
e = df_reviews.merge(anio_lanzamiento_item, on='reviews_item_id')


In [54]:
e

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis,año_lanzamiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,2,2009
1,EndAtHallow,http://steamcommunity.com/id/EndAtHallow,"Posted January 15, 2015.",1250,No ratings yet,True,2,2009
2,76561198107847795,http://steamcommunity.com/profiles/76561198107...,"Posted December 12, 2014.",1250,No ratings yet,True,1,2009
3,usaidwotnow,http://steamcommunity.com/id/usaidwotnow,"Posted December 13, 2013.",1250,0 of 1 people (0%) found this review helpful,True,0,2009
4,76561198081529182,http://steamcommunity.com/profiles/76561198081...,"Posted August 19, 2014.",1250,2 of 3 people (67%) found this review helpful,True,2,2009
...,...,...,...,...,...,...,...,...
50213,julez225,http://steamcommunity.com/id/julez225,"Posted May 17, 2012.",73010,No ratings yet,True,0,2011
50214,76561198063128416,http://steamcommunity.com/profiles/76561198063...,Posted June 13.,378930,3 of 17 people (18%) found this review helpful,False,1,2016
50215,76561198029064257,http://steamcommunity.com/profiles/76561198029...,"Posted October 22, 2012.",16600,No ratings yet,True,2,2008
50216,Darkjet15,http://steamcommunity.com/id/Darkjet15,"Posted January 19, 2014.",232950,No ratings yet,True,0,2013


### Conversion a csv y parquet

In [55]:
e.to_csv('345e.csv', index=False, encoding='utf-8')
pq.write_table(pa.Table.from_pandas(e), '3450e.parquet')